In this notebook, we create training data and test data consist of
* map pick, team composition
* players_stats
* economy per rounds, eco_ratings and
* special kills per rounds, special kill ratings.

In the end, we save them as csv files at "../data/vct_2023/processed_data/".

In [1]:
import pandas as pd

In [2]:
og_maps_scores = pd.read_csv("../data/vct_2023/matches/maps_scores.csv")
og_overview = pd.read_csv("../data/vct_2023/matches/overview.csv")
og_players_stats = pd.read_csv("../data/vct_2023/players_stats/players_stats.csv")

In [3]:
roles = {"duelist": {"jett", "phoenix", "reyna", "raze", "yoru", "neon", "iso"},  # dictionary of agents grouped by each role.
             "initiator": {"sova", "breach", "skye", "kayo", "fade", "gekko"},
             "controller": {"brimstone", "omen", "viper", "astra", "harbor"},
             "sentinel": {"cypher", "sage", "killjoy", "chamber", "deadlock"}}
allagents = set()  # initiate an empty set to make the set of all agents.
for role, l in roles.items():
    allagents = allagents.union(set(l))

duelists = roles["duelist"]  # set of all duelists
initiators = roles["initiator"]  # set of all initiators
controllers = roles["controller"]  # set of all controllers
sentinels = roles["sentinel"]  # set of all sentinels

def get_role(s:str) -> str:
    """
    input: agent names
    return: agent roles
    """
    lst = s.split(", ")
    num_of_agents = len(lst)
    output = ""
    if len(lst) == 1:
        agent = lst[0]
        if agent in duelists:
            output += "duel"
        elif agent in initiators:
            output += "init"
        elif agent in controllers:
            output += "cont"
        elif agent in sentinels:
            output += "sent"
        elif agent == "all":
            output += "agent"
        else:
            raise ValueError(f"{s} is not a valid agent.")
        return output
    else:
        for agent in lst:
            if agent in duelists:
                output += "duel,"
            elif agent in initiators:
                output += "init,"
            elif agent in controllers:
                output += "cont,"
            elif agent in sentinels:
                output += "sent,"
            elif agent == "all":
                output += "agent,"
            else:
                raise ValueError(f"{s} is not a valid agent.")
        return output[:-1]

In [4]:
og_players_stats.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10516 entries, 0 to 10515
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Tournament                      10516 non-null  object 
 1   Stage                           10516 non-null  object 
 2   Match Type                      10516 non-null  object 
 3   Player                          10516 non-null  object 
 4   Team                            10516 non-null  object 
 5   Agents                          10516 non-null  object 
 6   Rounds Played                   10516 non-null  int64  
 7   Rating                          9909 non-null   float64
 8   Average Combat Score            10504 non-null  float64
 9   Kills:Deaths                    10516 non-null  float64
 10  Kill, Assist, Trade, Survive %  9909 non-null   object 
 11  Average Damage Per Round        9909 non-null   float64
 12  Kills Per Round                 

In [5]:
og_players_stats.isna().sum()

Tournament                           0
Stage                                0
Match Type                           0
Player                               0
Team                                 0
Agents                               0
Rounds Played                        0
Rating                             607
Average Combat Score                12
Kills:Deaths                         0
Kill, Assist, Trade, Survive %     607
Average Damage Per Round           607
Kills Per Round                      0
Assists Per Round                    0
First Kills Per Round              584
First Deaths Per Round             607
Headshot %                         608
Clutch Success %                  5733
Clutches (won/played)             1170
Maximum Kills in a Single Map        0
Kills                                0
Deaths                               0
Assists                              0
First Kills                          0
First Deaths                         0
dtype: int64

In [6]:
match_id_columns = ["Tournament", "Stage", "Match Type", "Team"]
player_agent = ["Player", "Agents"]
features = ["Rating",
            "Average Combat Score",
            "Kill, Assist, Trade, Survive %",
            "Average Damage Per Round",
            "Kills Per Round",
            "Assists Per Round",
            "First Kills Per Round",
            "First Deaths Per Round",
            "Headshot %",
            "Clutch Success %"]
            #"Kills:Deaths", I will not use "Kills:Deaths" which is "First Kills Per Round" - "First Deaths Per Round".
            # Instead we kept "First Kills Per Round" and "First Deaths Per Round"
            #"Clutches (won/played)", this info is included in "Clutch Success %"
            #"Maximum Kills in a Single Map", other kill related features should reflect this information
            #"Kills", 
            #"Deaths",
            #"Assists",
            #"First Kills",
            #"First Deaths"
            # In many cases, one player played the same agent in two or three different maps in a match,
            # where "Kills", "Deaths", "Assists", "First Kills", "First Deaths" values were accumulated.
            # That's why I want to drop these columns.                 

In [7]:
players_stats = og_players_stats[match_id_columns + player_agent + features]
players_stats.head()

,Tournament,Stage,Match Type,Team,Player,Agents,Rating,Average Combat Score,"Kill, Assist, Trade, Survive %",Average Damage Per Round,Kills Per Round,Assists Per Round,First Kills Per Round,First Deaths Per Round,Headshot %,Clutch Success %
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,KRÜ Esports,Melser,brimstone,1.14,256.0,75%,168.9,0.85,0.45,0.10,0.15,32%,NaN
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,KRÜ Esports,Melser,omen,1.12,216.0,81%,131.8,0.86,0.05,0.19,0.05,25%,33%
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,KRÜ Esports,Melser,"brimstone, omen",1.13,236.0,78%,149.9,0.85,0.24,0.15,0.10,28%,25%
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,KRÜ Esports,DaveeyS,killjoy,1.29,253.0,86%,170.1,0.90,0.33,0.10,0.00,27%,100%
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,KRÜ Esports,keznit,raze,1.24,262.5,80%,172.5,0.85,0.39,0.15,0.10,15%,NaN


In [8]:
players_stats.keys()

Index(['Tournament', 'Stage', 'Match Type', 'Team', 'Player', 'Agents',
       'Rating', 'Average Combat Score', 'Kill, Assist, Trade, Survive %',
       'Average Damage Per Round', 'Kills Per Round', 'Assists Per Round',
       'First Kills Per Round', 'First Deaths Per Round', 'Headshot %',
       'Clutch Success %'],
      dtype='object')

In [9]:
# make numeric features to be actually numeric

numeric_columns = ["Kill, Assist, Trade, Survive %", "Headshot %", "Clutch Success %"]
for feature in numeric_columns:
    players_stats[feature] = players_stats[feature].apply(lambda s: int(s.split("%")[0])/100 if type(s) == str else None)

players_stats.info()  # "Numeric" features are now really numeric.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10516 entries, 0 to 10515
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Tournament                      10516 non-null  object 
 1   Stage                           10516 non-null  object 
 2   Match Type                      10516 non-null  object 
 3   Team                            10516 non-null  object 
 4   Player                          10516 non-null  object 
 5   Agents                          10516 non-null  object 
 6   Rating                          9909 non-null   float64
 7   Average Combat Score            10504 non-null  float64
 8   Kill, Assist, Trade, Survive %  9909 non-null   float64
 9   Average Damage Per Round        9909 non-null   float64
 10  Kills Per Round                 10516 non-null  float64
 11  Assists Per Round               10516 non-null  float64
 12  First Kills Per Round           

/var/folders/5l/9ggb45sn6nzb744shb8z4c680000gn/T/ipykernel_79197/1421638827.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_stats[feature] = players_stats[feature].apply(lambda s: int(s.split("%")[0])/100 if type(s) == str else None)


In [10]:
players_stats.isna().sum()

Tournament                           0
Stage                                0
Match Type                           0
Team                                 0
Player                               0
Agents                               0
Rating                             607
Average Combat Score                12
Kill, Assist, Trade, Survive %     607
Average Damage Per Round           607
Kills Per Round                      0
Assists Per Round                    0
First Kills Per Round              584
First Deaths Per Round             607
Headshot %                         608
Clutch Success %                  5733
dtype: int64

In [11]:
columns_with_missing_values = ["Rating",
                                "Average Combat Score",
                                "Kill, Assist, Trade, Survive %",
                                "Average Damage Per Round",
                                "First Kills Per Round",
                                "First Deaths Per Round",
                                "Headshot %",
                                "Clutch Success %"]

In [12]:
# I will fill in missing values with average of each player's performance of each tournament and then each team's performance and so on.
# See the following cells to check how I fillin missing values.
for column in columns_with_missing_values:
    players_stats.loc[:,column] = players_stats\
        .groupby(["Tournament", "Player"])[column].transform(lambda x: x.fillna(x.mean()))

In [13]:
for column in columns_with_missing_values:
    players_stats.loc[:,column] = players_stats\
    .groupby(["Tournament", "Team"])[column].transform(lambda x: x.fillna(x.mean()))

In [14]:
for column in columns_with_missing_values:
    players_stats.loc[:,column] = players_stats\
        .groupby(["Player", "Agents"])[column].transform(lambda x: x.fillna(x.mean()))

In [15]:
for column in columns_with_missing_values:
    players_stats.loc[:,column] = players_stats\
        .groupby(["Player"])[column].transform(lambda x: x.fillna(x.mean()))

In [16]:
for column in columns_with_missing_values:
    players_stats.loc[:,column] = players_stats\
        .groupby(["Team"])[column].transform(lambda x: x.fillna(x.mean()))

In [17]:
players_stats.isna().sum().sum()

0

In [18]:
ind_side_is_both = og_overview["Side"] == "both"
ind_map = og_overview["Map"] != "All Maps"

overview_agent_pick_keys = ["Tournament", "Stage", "Match Type", "Match Name", "Map", "Player", 'Team', 'Agents']
agent_pick_from_overview = og_overview[ind_side_is_both & ind_map][overview_agent_pick_keys]

agent_pick_from_overview

,Tournament,Stage,Match Type,Match Name,Map,Player,Team,Agents
0,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,nAts,Team Liquid,viper
3,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Sayf,Team Liquid,breach
6,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,soulcas,Team Liquid,astra
9,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Jamppi,Team Liquid,neon
12,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Redgar,Team Liquid,sova
...,...,...,...,...,...,...,...,...
34929,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Icebox,Derke,FNATIC,jett
34932,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Icebox,Boaster,FNATIC,viper
34935,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Icebox,Alfajer,FNATIC,killjoy
34938,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Icebox,Leo,FNATIC,sova


In [19]:
merge_keys = ["Tournament", "Stage", "Match Type", "Player", "Agents", "Team"]
players_stats_agent_pick = agent_pick_from_overview\
                                .merge(players_stats, on=merge_keys, how="left")\
                                

# Run the line below if you want to see where we have missing players stats.                                
#           |
#           V
# players_stats_agent_pick[players_stats_agent_pick.isna().any(axis=1)].groupby(["Tournament", "Stage", "Match Type", "Team"]).agg("count")

In [20]:
players_stats_agent_pick_columns_with_null\
      = ["Rating",
        "Average Combat Score",
        "Kill, Assist, Trade, Survive %",
        "Average Damage Per Round",
        "Kills Per Round",
        "Assists Per Round",
        "First Kills Per Round",
        "First Deaths Per Round",
        "Headshot %",
        "Clutch Success %"]

In [21]:
# There are 586 missing values from players_stats file.
# I checked the vlr.gg if the website has those missing values.
# At least I confirmed that it has some missing values.
# Because I don't have enough time to scrape the data,
# I will just take average of player's exisiting stats.
# There can be many different ways to deal with null values, but I chose to do in the following way.

for column in players_stats_agent_pick_columns_with_null:
    players_stats_agent_pick.loc[:,column] = players_stats_agent_pick\
        .groupby(["Tournament", "Stage", "Match Type", "Team"])[column]\
            .transform(lambda x: x.fillna(x.mean()))\
            

In [22]:
for column in players_stats_agent_pick_columns_with_null:
    players_stats_agent_pick.loc[:,column] = players_stats_agent_pick\
        .groupby(["Tournament", "Stage", "Team"])[column].transform(lambda x: x.fillna(x.mean()))

In [23]:
players_stats_agent_pick.isna().sum().sum()

0

In [24]:
players_stats_agent_pick = players_stats_agent_pick.round(2)  # Decimal digits were too big.

Next step is to combine "players_stats_by_agents_and_team_composition" with map results.

In [25]:
og_maps_scores.head(3)

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team A Score,Team A Attacker Score,Team A Defender Score,Team A Overtime Score,Team B,Team B Score,Team B Attacker Score,Team B Defender Score,Team B Overtime Score,Duration
0,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Team Liquid,11,6,5,NaN,Natus Vincere,13,7,6,NaN,1:18:55
1,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Bind,Team Liquid,15,7,5,3.0,Natus Vincere,17,7,5,5.0,1:22:57
2,Valorant Champions 2023,Group Stage,Opening (D),DRX vs LOUD,Lotus,DRX,13,7,5,1.0,LOUD,15,7,5,3.0,1:17:19


In [26]:
# Extracting map results and score difference.

Team_A_win = (og_maps_scores["Team A Score"] > og_maps_scores["Team B Score"])
Team_B_win = (og_maps_scores["Team B Score"] > og_maps_scores["Team A Score"])
Team_A_score_diff = og_maps_scores["Team A Score"] - og_maps_scores["Team B Score"]
Team_B_score_diff = og_maps_scores["Team B Score"] - og_maps_scores["Team A Score"]

map_results_with_score_diff = pd.DataFrame({
"Team_A_win": Team_A_win,
"Team_B_win": Team_B_win,
"Team_A_score_diff": Team_A_score_diff,
"Team_B_score_diff": Team_B_score_diff
})

In [27]:
maps_scores_results = pd.concat([og_maps_scores, map_results_with_score_diff], axis=1)

In [28]:
maps_scores_results.keys()

Index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A',
       'Team A Score', 'Team A Attacker Score', 'Team A Defender Score',
       'Team A Overtime Score', 'Team B', 'Team B Score',
       'Team B Attacker Score', 'Team B Defender Score',
       'Team B Overtime Score', 'Duration', 'Team_A_win', 'Team_B_win',
       'Team_A_score_diff', 'Team_B_score_diff'],
      dtype='object')

In [29]:
# For now, I don't have a plan to separately analyze the first half and the second half.
# I also will not pay attention to overtime, so my main features will be the following.
maps_scores_results_main_features = [\
    "Team_A_win", "Team_A_score_diff", "Team_B_win", "Team_B_score_diff"\
      ]\

map_outcomes_AvsB\
    = maps_scores_results[\
        ["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team A", "Team B"]\
        + maps_scores_results_main_features
        ]\
        


In [30]:
print(f"The size of map_outcome {len(map_outcomes_AvsB)} times 10 (5 players each from two teams) is")
print(f"the size of players_stats_agent_pick {len(players_stats_agent_pick)}, so it's all good!")

The size of map_outcome 830 times 10 (5 players each from two teams) is
the size of players_stats_agent_pick 8300, so it's all good!


In [31]:
# Let's add "Role" column to "players_stats_agent_pick".  This column indicates a role of an agent in each row.
# Remark: get_role function is more complicated than what we need here because I want to use it in different situation as well.
players_stats_agent_pick["Role"] = players_stats_agent_pick["Agents"].apply(get_role)

In [32]:
composition_dummies = pd.get_dummies(players_stats_agent_pick["Role"], dtype=int)
players_stats_team_comp = pd.concat([players_stats_agent_pick, composition_dummies], axis=1).round(2)

In [33]:
print(f"The size of players_stats_team_comp is {len(players_stats_team_comp)}.")
players_stats_team_comp.head(2)

The size of players_stats_team_comp is 8300.


,Tournament,Stage,Match Type,Match Name,Map,Player,Team,Agents,Rating,Average Combat Score,...,Assists Per Round,First Kills Per Round,First Deaths Per Round,Headshot %,Clutch Success %,Role,cont,duel,init,sent
0,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,nAts,Team Liquid,viper,1.26,263.0,...,0.18,0.14,0.09,0.31,0.27,cont,1,0,0,0
1,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Sayf,Team Liquid,breach,0.96,170.0,...,0.42,0.00,0.04,0.23,0.33,init,0,0,1,0


In [34]:
players_stats_team_comp.keys()

Index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Player',
       'Team', 'Agents', 'Rating', 'Average Combat Score',
       'Kill, Assist, Trade, Survive %', 'Average Damage Per Round',
       'Kills Per Round', 'Assists Per Round', 'First Kills Per Round',
       'First Deaths Per Round', 'Headshot %', 'Clutch Success %', 'Role',
       'cont', 'duel', 'init', 'sent'],
      dtype='object')

In [35]:
# If we use deep learning models, we probably don't need to do what I did in this cell.

stats_list = ['Rating', 'Average Combat Score',
       'Kill, Assist, Trade, Survive %', 'Average Damage Per Round',
       'Kills Per Round', 'Assists Per Round', 'First Kills Per Round',
       'First Deaths Per Round', 'Headshot %', 'Clutch Success %']
role_list = ['cont', 'duel', 'init', 'sent']

role_stats_dict = {}  # initiate a dictionary.  By multiplying "duel", "init",... with "Rating", "Average Combat Score", ...
                        # we have each role's stats of each team and each game/map

for stat in stats_list:
    for role in role_list:
        role_stats_dict[role+"_"+stat] = players_stats_team_comp[stat] * players_stats_team_comp[role]


In [36]:
# # After adding role_stats_dict, we can drop the following columns.
# columns_to_drop = ['Rating', 'Average Combat Score',
#        'Kill, Assist, Trade, Survive %', 'Average Damage Per Round',
#        'Kills Per Round', 'Assists Per Round', 'First Kills Per Round',
#        'First Deaths Per Round', 'Headshot %', 'Clutch Success %', 'Role']

# players_stats_team_comp_before_agg = pd.concat([players_stats_team_comp, pd.DataFrame(role_stats_dict)], axis=1).drop(columns_to_drop, axis=1)

players_stats_team_comp_before_agg = pd.concat([players_stats_team_comp, pd.DataFrame(role_stats_dict)], axis=1).drop(["Role"], axis=1)

In [37]:
players_stats_team_comp_before_agg

,Tournament,Stage,Match Type,Match Name,Map,Player,Team,Agents,Rating,Average Combat Score,...,init_First Deaths Per Round,sent_First Deaths Per Round,cont_Headshot %,duel_Headshot %,init_Headshot %,sent_Headshot %,cont_Clutch Success %,duel_Clutch Success %,init_Clutch Success %,sent_Clutch Success %
0,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,nAts,Team Liquid,viper,1.26,263.00,...,0.00,0.00,0.31,0.00,0.00,0.00,0.27,0.00,0.00,0.00
1,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Sayf,Team Liquid,breach,0.96,170.00,...,0.04,0.00,0.00,0.00,0.23,0.00,0.00,0.00,0.33,0.00
2,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,soulcas,Team Liquid,astra,0.95,152.00,...,0.00,0.00,0.36,0.00,0.00,0.00,0.17,0.00,0.00,0.00
3,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Jamppi,Team Liquid,neon,0.89,194.00,...,0.00,0.00,0.00,0.17,0.00,0.00,0.00,0.37,0.00,0.00
4,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Redgar,Team Liquid,sova,0.69,125.00,...,0.13,0.00,0.00,0.00,0.20,0.00,0.00,0.00,0.27,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8295,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Icebox,Derke,FNATIC,jett,1.10,264.30,...,0.00,0.00,0.00,0.21,0.00,0.00,0.00,0.33,0.00,0.00
8296,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Icebox,Boaster,FNATIC,viper,0.95,180.00,...,0.00,0.00,0.21,0.00,0.00,0.00,0.48,0.00,0.00,0.00
8297,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Icebox,Alfajer,FNATIC,killjoy,0.96,178.00,...,0.00,0.07,0.00,0.00,0.00,0.29,0.00,0.00,0.00,0.23
8298,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Icebox,Leo,FNATIC,sova,0.97,200.36,...,0.12,0.00,0.00,0.00,0.23,0.00,0.00,0.00,0.30,0.00


In [38]:
# I will use this function to get team composition.
def sort_join(x):
    """
    input: array of strings
    output: sorted strings joined by ","
    """
    return ",".join(sorted(list(x)))


In [39]:
left =\
    players_stats_team_comp_before_agg.groupby(["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"]).agg({\
        "Player": sort_join,
        "Agents": sort_join
        })\
        .reset_index()\
        .rename(columns={"Agents":"Composition"})

right =\
    players_stats_team_comp_before_agg.drop(["Player", "Agents"], axis=1)\
    .groupby(["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"]).agg("sum")\
    .reset_index()


In [40]:
merge_keys_2 = ["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"]
stats_team_comp = pd.merge(left, right, on=merge_keys_2)  # This has both players stats and team composition data.

stats_team_comp

,Tournament,Stage,Match Type,Match Name,Map,Team,Player,Composition,Rating,Average Combat Score,...,init_First Deaths Per Round,sent_First Deaths Per Round,cont_Headshot %,duel_Headshot %,init_Headshot %,sent_Headshot %,cont_Clutch Success %,duel_Clutch Success %,init_Clutch Success %,sent_Clutch Success %
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,KRÜ Esports,"DaveeyS,Klaus,Melser,NagZ,keznit","jett,kayo,killjoy,omen,sova",5.58,1029.00,...,0.26,0.00,0.28,0.20,0.49,0.17,0.25,0.17,1.50,0.25
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,Leviatán,"Mazino,Shyy,Tacolilla,kiNgg,nzr","jett,kayo,killjoy,omen,sova",4.65,939.80,...,0.16,0.08,0.18,0.17,0.61,0.22,0.25,0.33,0.49,0.25
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Lotus,KRÜ Esports,"DaveeyS,Klaus,Melser,NagZ,keznit","killjoy,omen,raze,skye,viper",5.19,925.00,...,0.11,0.11,0.60,0.14,0.17,0.28,0.45,0.34,0.43,0.13
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Lotus,Leviatán,"Mazino,Shyy,Tacolilla,kiNgg,nzr","fade,killjoy,omen,raze,viper",4.62,972.34,...,0.11,0.08,0.42,0.13,0.22,0.22,0.36,0.20,0.27,0.25
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Pearl,KRÜ Esports,"DaveeyS,Klaus,Melser,NagZ,keznit","harbor,jett,killjoy,skye,viper",5.27,1030.50,...,0.11,0.11,0.70,0.20,0.17,0.28,0.56,0.17,0.43,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655,Valorant Champions 2023,Playoffs,Upper Semifinals,LOUD vs Paper Rex,Lotus,Paper Rex,"Jinggg,d4v41,f0rsakeN,mindfreak,something","astra,jett,omen,raze,skye",4.41,928.50,...,0.02,0.00,0.49,0.41,0.37,0.00,0.47,0.47,0.20,0.00
1656,Valorant Champions 2023,Playoffs,Upper Semifinals,LOUD vs Paper Rex,Pearl,LOUD,"Less,aspas,cauanzin,saadhak,tuyz","astra,breach,fade,jett,killjoy",5.94,1097.80,...,0.19,0.03,0.35,0.40,0.47,0.24,0.32,0.28,0.56,0.14
1657,Valorant Champions 2023,Playoffs,Upper Semifinals,LOUD vs Paper Rex,Pearl,Paper Rex,"Jinggg,d4v41,f0rsakeN,mindfreak,something","astra,jett,killjoy,phoenix,skye",4.85,942.07,...,0.02,0.09,0.23,0.41,0.37,0.18,0.17,0.44,0.20,0.33
1658,Valorant Champions 2023,Playoffs,Upper Semifinals,LOUD vs Paper Rex,Split,LOUD,"Less,aspas,cauanzin,saadhak,tuyz","astra,breach,killjoy,raze,skye",5.81,1005.30,...,0.18,0.03,0.35,0.23,0.42,0.24,0.32,0.28,0.62,0.14


In [41]:
stats_team_comp.keys()

Index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team',
       'Player', 'Composition', 'Rating', 'Average Combat Score',
       'Kill, Assist, Trade, Survive %', 'Average Damage Per Round',
       'Kills Per Round', 'Assists Per Round', 'First Kills Per Round',
       'First Deaths Per Round', 'Headshot %', 'Clutch Success %', 'cont',
       'duel', 'init', 'sent', 'cont_Rating', 'duel_Rating', 'init_Rating',
       'sent_Rating', 'cont_Average Combat Score', 'duel_Average Combat Score',
       'init_Average Combat Score', 'sent_Average Combat Score',
       'cont_Kill, Assist, Trade, Survive %',
       'duel_Kill, Assist, Trade, Survive %',
       'init_Kill, Assist, Trade, Survive %',
       'sent_Kill, Assist, Trade, Survive %', 'cont_Average Damage Per Round',
       'duel_Average Damage Per Round', 'init_Average Damage Per Round',
       'sent_Average Damage Per Round', 'cont_Kills Per Round',
       'duel_Kills Per Round', 'init_Kills Per Round', 'sent_Kills Per

In [42]:
# Let's add special kill information to stats_team_comp

special_kill_rating = pd.read_csv("../data/vct_2023/special_kills_non_finals.csv").drop(['Unnamed: 0'], axis=1)
special_kill_rounds = pd.read_csv("../data/vct_2023/special_kills_non_finals_rounds.csv").drop(['Unnamed: 0'], axis=1)
eco_ratings = pd.read_csv("../data/vct_2023/eco_ratings.csv").drop(['Unnamed: 0'], axis=1)

In [43]:
special_kill_rating.keys()

Index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Eliminator Team',
       'Map', 'Won Match', 'SK_Rating'],
      dtype='object')

In [44]:
# Change feature names so that their names coincide with stats_team_comp feature names.

special_kill_rating = special_kill_rating.rename(columns={"Eliminator Team": "Team"}).drop("Won Match", axis=1)
special_kill_rounds = special_kill_rounds.rename(columns={"Eliminator Team": "Team"}).drop("Won Match", axis=1)

In [45]:
special_kill_rating.keys()

Index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Team', 'Map',
       'SK_Rating'],
      dtype='object')

In [46]:
eco_ratings.keys()

Index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team B',
       'Team A', 'Team_A_My_Eco_Rating', 'Team_B_My_Eco_Rating', 'R1_A_lo',
       'R1_B_lo', 'R2_A_lo', 'R2_B_lo', 'R3_A_lo', 'R3_B_lo', 'R4_A_lo',
       'R4_B_lo', 'R5_A_lo', 'R5_B_lo', 'R6_A_lo', 'R6_B_lo', 'R7_A_lo',
       'R7_B_lo', 'R8_A_lo', 'R8_B_lo', 'R9_A_lo', 'R9_B_lo', 'R10_A_lo',
       'R10_B_lo', 'R11_A_lo', 'R11_B_lo', 'R12_A_lo', 'R12_B_lo', 'R13_A_lo',
       'R13_B_lo', 'Team_A_win', 'Team_B_win', 'Team_A_Prob_Eco_Rating',
       'Team_B_Prob_Eco_Rating'],
      dtype='object')

In [47]:
teamA_features = ['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map',
    'Team A', 'Team_A_My_Eco_Rating', 'Team_A_Prob_Eco_Rating',
       'R1_A_lo', 'R2_A_lo', 'R3_A_lo', 
       'R4_A_lo', 'R5_A_lo', 'R6_A_lo',
       'R7_A_lo', 'R8_A_lo', 'R9_A_lo',
       'R10_A_lo', 'R11_A_lo', 'R12_A_lo',
       'R13_A_lo']
teamB_features = ['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map',
    'Team B', 'Team_B_My_Eco_Rating', 'Team_B_Prob_Eco_Rating',
       'R1_B_lo', 'R2_B_lo', 'R3_B_lo', 
       'R4_B_lo', 'R5_B_lo', 'R6_B_lo',
       'R7_B_lo', 'R8_B_lo', 'R9_B_lo',
       'R10_B_lo', 'R11_B_lo', 'R12_B_lo',
       'R13_B_lo']

In [48]:
TeamA_eco_ratings = eco_ratings[teamA_features].rename(
    columns={
        'Team A':"Team", 'Team_A_My_Eco_Rating':"eco_rating", 'Team_A_Prob_Eco_Rating':"prob_eco_rating",
        'R1_A_lo':"R1_lo", 'R2_A_lo':"R2_lo", 'R3_A_lo':"R3_lo", 
        'R4_A_lo':"R4_lo", 'R5_A_lo':"R5_lo", 'R6_A_lo':"R6_lo",
        'R7_A_lo':"R7_lo", 'R8_A_lo':"R8_lo", 'R9_A_lo':"R9_lo",
        'R10_A_lo':"R10_lo", 'R11_A_lo':"R11_lo", 'R12_A_lo':"R12_lo",
        'R13_A_lo':"R13_lo"
    }
)

TeamB_eco_ratings = eco_ratings[teamB_features].rename(
    columns={
        'Team B':"Team", 'Team_B_My_Eco_Rating':"eco_rating", 'Team_B_Prob_Eco_Rating':"prob_eco_rating",
        'R1_B_lo':"R1_lo", 'R2_B_lo':"R2_lo", 'R3_B_lo':"R3_lo", 
        'R4_B_lo':"R4_lo", 'R5_B_lo':"R5_lo", 'R6_B_lo':"R6_lo",
        'R7_B_lo':"R7_lo", 'R8_B_lo':"R8_lo", 'R9_B_lo':"R9_lo",
        'R10_B_lo':"R10_lo", 'R11_B_lo':"R11_lo", 'R12_B_lo':"R12_lo",
        'R13_B_lo':"R13_lo"
    }
)

In [49]:
stacked_eco_ratings = pd.concat([TeamA_eco_ratings,TeamB_eco_ratings])

In [50]:
stacked_eco_ratings

,Tournament,Stage,Match Type,Match Name,Map,Team,eco_rating,prob_eco_rating,R1_lo,R2_lo,...,R4_lo,R5_lo,R6_lo,R7_lo,R8_lo,R9_lo,R10_lo,R11_lo,R12_lo,R13_lo
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Bind,MIBR,-0.244332,0.083456,3900,4300,...,7500,22600,10600,23300,19100,21900,23900,11700,20800,3800
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Lotus,MIBR,-0.150451,0.063740,3400,2800,...,21200,21700,19000,9100,21500,16800,19700,5800,19900,4300
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Split,FURIA,0.053344,0.717766,4000,16000,...,20400,19500,21000,23100,23200,23200,22400,20900,23600,4100
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Ascent,FURIA,-0.831858,0.225370,3600,15000,...,21500,7800,21800,19900,21800,22400,10100,22200,14400,3700
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Bind,FURIA,0.221960,0.430977,3800,1500,...,7000,25000,25000,25300,22400,19300,11100,23100,24600,4200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Ascent,FNATIC,0.242662,0.749069,3200,17600,...,5100,19500,19800,20300,23000,23700,25000,22500,23200,3500
684,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Fracture,FNATIC,0.000640,0.598076,3900,14900,...,16100,20900,22100,21200,21800,18100,13700,20000,19300,3600
685,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Split,FNATIC,-0.132743,0.439970,3800,15400,...,19500,17700,20200,16700,18200,20600,12500,19500,22700,3900
686,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Lotus,FNATIC,-0.166363,0.198987,3600,15200,...,24000,18200,10400,18800,10700,19000,18000,21000,20200,3600


In [51]:
stats_team_comp

,Tournament,Stage,Match Type,Match Name,Map,Team,Player,Composition,Rating,Average Combat Score,...,init_First Deaths Per Round,sent_First Deaths Per Round,cont_Headshot %,duel_Headshot %,init_Headshot %,sent_Headshot %,cont_Clutch Success %,duel_Clutch Success %,init_Clutch Success %,sent_Clutch Success %
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,KRÜ Esports,"DaveeyS,Klaus,Melser,NagZ,keznit","jett,kayo,killjoy,omen,sova",5.58,1029.00,...,0.26,0.00,0.28,0.20,0.49,0.17,0.25,0.17,1.50,0.25
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,Leviatán,"Mazino,Shyy,Tacolilla,kiNgg,nzr","jett,kayo,killjoy,omen,sova",4.65,939.80,...,0.16,0.08,0.18,0.17,0.61,0.22,0.25,0.33,0.49,0.25
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Lotus,KRÜ Esports,"DaveeyS,Klaus,Melser,NagZ,keznit","killjoy,omen,raze,skye,viper",5.19,925.00,...,0.11,0.11,0.60,0.14,0.17,0.28,0.45,0.34,0.43,0.13
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Lotus,Leviatán,"Mazino,Shyy,Tacolilla,kiNgg,nzr","fade,killjoy,omen,raze,viper",4.62,972.34,...,0.11,0.08,0.42,0.13,0.22,0.22,0.36,0.20,0.27,0.25
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Pearl,KRÜ Esports,"DaveeyS,Klaus,Melser,NagZ,keznit","harbor,jett,killjoy,skye,viper",5.27,1030.50,...,0.11,0.11,0.70,0.20,0.17,0.28,0.56,0.17,0.43,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655,Valorant Champions 2023,Playoffs,Upper Semifinals,LOUD vs Paper Rex,Lotus,Paper Rex,"Jinggg,d4v41,f0rsakeN,mindfreak,something","astra,jett,omen,raze,skye",4.41,928.50,...,0.02,0.00,0.49,0.41,0.37,0.00,0.47,0.47,0.20,0.00
1656,Valorant Champions 2023,Playoffs,Upper Semifinals,LOUD vs Paper Rex,Pearl,LOUD,"Less,aspas,cauanzin,saadhak,tuyz","astra,breach,fade,jett,killjoy",5.94,1097.80,...,0.19,0.03,0.35,0.40,0.47,0.24,0.32,0.28,0.56,0.14
1657,Valorant Champions 2023,Playoffs,Upper Semifinals,LOUD vs Paper Rex,Pearl,Paper Rex,"Jinggg,d4v41,f0rsakeN,mindfreak,something","astra,jett,killjoy,phoenix,skye",4.85,942.07,...,0.02,0.09,0.23,0.41,0.37,0.18,0.17,0.44,0.20,0.33
1658,Valorant Champions 2023,Playoffs,Upper Semifinals,LOUD vs Paper Rex,Split,LOUD,"Less,aspas,cauanzin,saadhak,tuyz","astra,breach,killjoy,raze,skye",5.81,1005.30,...,0.18,0.03,0.35,0.23,0.42,0.24,0.32,0.28,0.62,0.14


In [52]:
keys = ['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team']
stats_team_comp_with_eco_SK = stats_team_comp.merge(special_kill_rating, on=keys, how="left")\
                                .merge(stacked_eco_ratings, on=keys, how="left")\
                                
stats_team_comp_with_eco_SK_rounds = stats_team_comp.merge(special_kill_rounds, on=keys, how="left")\
                                        .merge(stacked_eco_ratings, on=keys, how="left")\
                                        

In [53]:
stats_team_comp_with_eco_SK.isna().sum().sum()

4544

In [54]:
features_eco_SK = stats_team_comp_with_eco_SK.keys()
features_eco_SK_rounds = stats_team_comp_with_eco_SK_rounds.keys()
features_ignore = set(['Tournament', 'Stage', 'Match Type', 'Match Name','Composition', 'Win', 'Score_diff', 'Map', 'Team','Player'])

for feature in features_eco_SK:
    if feature not in features_ignore:
        stats_team_comp_with_eco_SK[feature] = stats_team_comp_with_eco_SK\
                            .drop(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Composition'],axis=1)\
                                .groupby(["Team"])[feature].transform(lambda x: x.fillna(x.mean()))
        
for feature in features_eco_SK_rounds:
    if feature not in features_ignore:
        stats_team_comp_with_eco_SK_rounds[feature] = stats_team_comp_with_eco_SK_rounds\
                            .drop(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Composition'],axis=1)\
                                .groupby(["Team"])[feature].transform(lambda x: x.fillna(x.mean()))

In [55]:
stats_team_comp_with_eco_SK_rounds.isna().sum().sum()

728

In [56]:
for feature in features_eco_SK:
    if feature not in features_ignore:
        mean_value = stats_team_comp_with_eco_SK[feature].mean()
        stats_team_comp_with_eco_SK[feature].fillna(value=mean_value, inplace=True)

for feature in features_eco_SK_rounds:
    if feature not in features_ignore:
        mean_value = stats_team_comp_with_eco_SK_rounds[feature].mean()
        stats_team_comp_with_eco_SK_rounds[feature].fillna(value=mean_value, inplace=True)

In [57]:
stats_team_comp_with_eco_SK_rounds.isna().sum().sum()

0

In [58]:
map_outcomes_A = map_outcomes_AvsB[['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A','Team_A_win', 'Team_A_score_diff']]\
        .rename(columns={"Team A": "Team", "Team_A_win": "Win", "Team_A_score_diff":"Score_diff"})
map_outcomes_B = map_outcomes_AvsB[['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team B','Team_B_win', 'Team_B_score_diff']]\
        .rename(columns={"Team B": "Team", "Team_B_win": "Win", "Team_B_score_diff":"Score_diff"})

In [59]:
map_outcomes = pd.concat([map_outcomes_A, map_outcomes_B])
# In this dataframe, we don't compare Team A and Team B.  Only look at one team of each map/game.
#      |
#      v
stats_team_comp_outcomes = pd.merge(stats_team_comp, map_outcomes, on=merge_keys_2)

In [60]:
# Same as the above cell but with special kills considered.

stats_eco_and_SK_outcomes = pd.merge(stats_team_comp_with_eco_SK, map_outcomes, on=merge_keys_2)
stats_eco_and_SK_rounds_outcomes = pd.merge(stats_team_comp_with_eco_SK_rounds, map_outcomes, on=merge_keys_2)

In [61]:
stats_eco_and_SK_rounds_outcomes.isna().sum().sum()

0

In [62]:
# Now I want to combine stats_team_comp and map_outcome_AvsB
# We can use the same merge keys, merge_keys_2 = ["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"]
# In this dataframe, we do "Team A" vs "Team B".
#       |
#       v
AvsB_stats_team_comp_outcomes = map_outcomes_AvsB\
                                    .rename(columns={"Team A":"Team"})\
                                    .merge(stats_team_comp, on=merge_keys_2)\
                                    .rename(columns={"Team": "Team_A"})\
                                    .rename(columns={"Team B":"Team"})\
                                    .merge(stats_team_comp, on=merge_keys_2, suffixes=("_A", "_B"))\
                                    .rename(columns={"Team": "Team_B"})\



In [63]:
# Same as the above cell but with special kills considered.

AvsB_stats_eco_and_SK_outcomes = map_outcomes_AvsB\
                                    .rename(columns={"Team A":"Team"})\
                                    .merge(stats_team_comp_with_eco_SK, on=merge_keys_2)\
                                    .rename(columns={"Team": "Team_A"})\
                                    .rename(columns={"Team B":"Team"})\
                                    .merge(stats_team_comp_with_eco_SK, on=merge_keys_2, suffixes=("_A", "_B"))\
                                    .rename(columns={"Team": "Team_B"})\
                                    

AvsB_stats_eco_and_SK_rounds_outcomes = map_outcomes_AvsB\
                                    .rename(columns={"Team A":"Team"})\
                                    .merge(stats_team_comp_with_eco_SK_rounds, on=merge_keys_2)\
                                    .rename(columns={"Team": "Team_A"})\
                                    .rename(columns={"Team B":"Team"})\
                                    .merge(stats_team_comp_with_eco_SK_rounds, on=merge_keys_2, suffixes=("_A", "_B"))\
                                    .rename(columns={"Team": "Team_B"})\
                                    

In [64]:
AvsB_stats_eco_and_SK_rounds_outcomes.isna().sum().sum()

0

In [65]:
# AvsB_stats_team_comp_outcomes

### To prevent data leakage.
- I will make "test_data" and "test_data2" containing players stats from earlier tournaments.\
Also, similarly "AvsB_test_data" and "AvsB_test_data2" which have "Team A" vs "Team B" structure.

- Players' stats in "test_data" and "AvsB_test_data" will be computed from smaller sets of tournaments than "test_data2" and "AvsB_test_data2".\
Their stats are based on the latest_tournaments that you can find in the following cell.

- Players' stats in "test_data2" and "AvsB_test_data2" will be computed from all tournaments but the last tournament which is "Valorant Champions 2023".

In [66]:
latest_tournaments = set(['Champions Tour 2023: Americas Last Chance Qualifier',
       'Champions Tour 2023: EMEA Last Chance Qualifier',
       'Champions Tour 2023: Pacific Last Chance Qualifier',
       'Champions Tour 2023: Champions China Qualifier',
       'Champions Tour 2023: Masters Tokyo'])
last_tournament = "Valorant Champions 2023"


In [67]:
train_data_stats_outcomes = stats_team_comp_outcomes[stats_team_comp_outcomes.Tournament != last_tournament]
train_data_stats_eco_SK_outcomes = stats_eco_and_SK_outcomes[stats_eco_and_SK_outcomes.Tournament != last_tournament]
train_data_stats_eco_SK_rounds_outcomes = stats_eco_and_SK_rounds_outcomes[stats_eco_and_SK_rounds_outcomes.Tournament != last_tournament]
# train_data_stats_outcomes2 = stats_team_comp_outcomes[stats_team_comp_outcomes.Tournament.isin(latest_tournaments)]
# When I have time later, I may play around with this more.

test_tournament = stats_team_comp_outcomes[stats_team_comp_outcomes.Tournament == last_tournament]
AvsB_test_tournament = map_outcomes_AvsB[map_outcomes_AvsB.Tournament == last_tournament]

In [68]:
train_data_stats_eco_SK_rounds_outcomes.isna().sum().sum()

0

In [69]:
AvsB_test_tournament.isna().sum().sum()

0

*Make these train data into Team A vs Team B format*

In [70]:
AvsB_train_stats_outcomes = AvsB_stats_team_comp_outcomes[AvsB_stats_team_comp_outcomes.Tournament != "Valorant Champions 2023"]
AvsB_train_stats_eco_and_SK_outcomes = AvsB_stats_eco_and_SK_outcomes[AvsB_stats_eco_and_SK_outcomes.Tournament != "Valorant Champions 2023"]
AvsB_train_stats_eco_and_SK_rounds_outcomes = AvsB_stats_eco_and_SK_rounds_outcomes[AvsB_stats_eco_and_SK_rounds_outcomes.Tournament != "Valorant Champions 2023"]


In [71]:
AvsB_train_stats_eco_and_SK_rounds_outcomes.isna().sum().sum()

0

*Modify train data so that we can use it as test input.*

In [72]:
data_from_earlier_tnmt =\
    train_data_stats_outcomes\
        .drop(['Tournament', 'Stage', 'Match Type', 'Match Name',"Player", "Composition", 'Win', 'Score_diff'], axis=1)\
            .groupby(["Map", "Team"])\
                .agg("mean")\
                .reset_index()\
            
data_from_earlier_tnmt

,Map,Team,Rating,Average Combat Score,"Kill, Assist, Trade, Survive %",Average Damage Per Round,Kills Per Round,Assists Per Round,First Kills Per Round,First Deaths Per Round,...,init_First Deaths Per Round,sent_First Deaths Per Round,cont_Headshot %,duel_Headshot %,init_Headshot %,sent_Headshot %,cont_Clutch Success %,duel_Clutch Success %,init_Clutch Success %,sent_Clutch Success %
0,Ascent,100 Thieves,4.452500,934.650000,3.362500,623.775000,3.185000,1.507500,0.497500,0.540000,...,0.157500,0.027500,0.475000,0.210000,0.492500,0.057500,0.507500,0.260000,0.670000,0.025000
1,Ascent,Attacking Soul Esports,4.665000,999.050000,3.335000,660.920000,3.450000,1.875000,0.465000,0.530000,...,0.230000,0.055000,0.300000,0.240000,0.430000,0.210000,0.260000,0.260000,0.400000,0.340000
2,Ascent,BBL Esports,5.236667,1005.500000,3.661667,656.650000,3.630000,1.595000,0.553333,0.485000,...,0.146667,0.090000,0.351667,0.283333,0.525000,0.266667,0.566667,0.241667,0.793333,0.323333
3,Ascent,Bilibili Gaming,4.770000,1092.680000,3.480000,605.780000,4.060000,1.460000,0.600000,0.530000,...,0.200000,0.090000,0.230000,0.210000,0.440000,0.270000,0.190000,0.110000,0.420000,0.250000
4,Ascent,Cloud9,5.217143,1019.871429,3.694286,670.900000,3.594286,1.665714,0.544286,0.465714,...,0.104286,0.062857,0.437143,0.228571,0.357143,0.192857,0.538571,0.567143,0.421429,0.222857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,Split,Team Vitality,5.395000,1043.536667,3.671667,680.643333,3.725000,1.430000,0.580000,0.471667,...,0.000000,0.215000,0.271667,0.435000,0.000000,0.596667,0.266667,0.573333,0.000000,0.688333
344,Split,Totoro Gaming,5.810000,1146.000000,3.850000,725.300000,3.990000,1.940000,0.600000,0.360000,...,0.200000,0.000000,0.370000,0.220000,0.460000,0.180000,0.300000,0.500000,0.790000,0.500000
345,Split,Trace Esports,5.910000,969.400000,3.950000,734.980000,3.453333,1.610000,0.580000,0.380000,...,0.250000,0.030000,0.270000,0.250000,0.470000,0.230000,0.320000,0.400000,0.760000,0.410000
346,Split,Weibo Gaming,5.376667,1098.166667,3.566667,675.253333,3.920000,1.406667,0.580000,0.530000,...,0.110000,0.073333,0.436667,0.273333,0.253333,0.153333,0.663333,0.423333,0.253333,0.203333


In [73]:
# data_from_earlier_tnmt2 =\
#     train_data_stats_outcomes2\
#         .drop(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Player', 'Composition', 'Win', 'Score_diff'], axis=1)\
#             .groupby(["Map", "Team"])\
#                 .agg("mean")\
#                 .reset_index()\
                
# data_from_earlier_tnmt2.head().round(2)

In [74]:
data_from_earlier_tnmt_eco_SK =\
    train_data_stats_eco_SK_outcomes\
        .drop(['Tournament', 'Stage', 'Match Type', 'Match Name',"Player", "Composition", 'Win', 'Score_diff'], axis=1)\
            .groupby(["Map", "Team"])\
                .agg("mean")\
                .reset_index()\
            
data_from_earlier_tnmt_eco_SK

,Map,Team,Rating,Average Combat Score,"Kill, Assist, Trade, Survive %",Average Damage Per Round,Kills Per Round,Assists Per Round,First Kills Per Round,First Deaths Per Round,...,R4_lo,R5_lo,R6_lo,R7_lo,R8_lo,R9_lo,R10_lo,R11_lo,R12_lo,R13_lo
0,Ascent,100 Thieves,4.452500,934.650000,3.362500,623.775000,3.185000,1.507500,0.497500,0.540000,...,19675.000000,20375.000000,18300.000000,17075.000000,20875.000000,20800.000000,14150.000000,22350.000000,20600.000000,4000.000000
1,Ascent,Attacking Soul Esports,4.665000,999.050000,3.335000,660.920000,3.450000,1.875000,0.465000,0.530000,...,18990.000000,19270.000000,21460.000000,19650.000000,10900.000000,21140.000000,21570.000000,18270.000000,23190.000000,3490.000000
2,Ascent,BBL Esports,5.236667,1005.500000,3.661667,656.650000,3.630000,1.595000,0.553333,0.485000,...,20466.666667,20033.333333,21233.333333,21083.333333,19616.666667,21433.333333,18733.333333,21833.333333,23466.666667,3650.000000
3,Ascent,Bilibili Gaming,4.770000,1092.680000,3.480000,605.780000,4.060000,1.460000,0.600000,0.530000,...,19445.534330,19031.889163,20330.259390,20374.227515,20283.434282,20779.470734,20682.918532,20887.386873,22423.192723,3792.172754
4,Ascent,Cloud9,5.217143,1019.871429,3.694286,670.900000,3.594286,1.665714,0.544286,0.465714,...,17085.714286,20385.714286,23285.714286,20714.285714,24014.285714,23957.142857,21400.000000,21585.714286,22285.714286,4028.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,Split,Team Vitality,5.395000,1043.536667,3.671667,680.643333,3.725000,1.430000,0.580000,0.471667,...,20750.000000,17400.000000,21600.000000,19133.333333,21250.000000,19200.000000,20350.000000,22233.333333,23033.333333,3666.666667
344,Split,Totoro Gaming,5.810000,1146.000000,3.850000,725.300000,3.990000,1.940000,0.600000,0.360000,...,21200.000000,23500.000000,22300.000000,23300.000000,23500.000000,24900.000000,23500.000000,23300.000000,23800.000000,3700.000000
345,Split,Trace Esports,5.910000,969.400000,3.950000,734.980000,3.453333,1.610000,0.580000,0.380000,...,19445.534330,19031.889163,20330.259390,20374.227515,20283.434282,20779.470734,20682.918532,20887.386873,22423.192723,3792.172754
346,Split,Weibo Gaming,5.376667,1098.166667,3.566667,675.253333,3.920000,1.406667,0.580000,0.530000,...,20107.407407,16311.111111,17418.518519,19518.518519,17840.740741,17074.074074,20588.888889,20485.185185,19266.666667,3948.148148


In [75]:
data_from_earlier_tnmt_eco_SK_rounds =\
    train_data_stats_eco_SK_rounds_outcomes\
        .drop(['Tournament', 'Stage', 'Match Type', 'Match Name',"Player", "Composition", 'Win', 'Score_diff'], axis=1)\
            .groupby(["Map", "Team"])\
                .agg("mean")\
                .reset_index()\
            
data_from_earlier_tnmt_eco_SK_rounds

,Map,Team,Rating,Average Combat Score,"Kill, Assist, Trade, Survive %",Average Damage Per Round,Kills Per Round,Assists Per Round,First Kills Per Round,First Deaths Per Round,...,R4_lo,R5_lo,R6_lo,R7_lo,R8_lo,R9_lo,R10_lo,R11_lo,R12_lo,R13_lo
0,Ascent,100 Thieves,4.452500,934.650000,3.362500,623.775000,3.185000,1.507500,0.497500,0.540000,...,19675.000000,20375.000000,18300.000000,17075.000000,20875.000000,20800.000000,14150.000000,22350.000000,20600.000000,4000.000000
1,Ascent,Attacking Soul Esports,4.665000,999.050000,3.335000,660.920000,3.450000,1.875000,0.465000,0.530000,...,18990.000000,19270.000000,21460.000000,19650.000000,10900.000000,21140.000000,21570.000000,18270.000000,23190.000000,3490.000000
2,Ascent,BBL Esports,5.236667,1005.500000,3.661667,656.650000,3.630000,1.595000,0.553333,0.485000,...,20466.666667,20033.333333,21233.333333,21083.333333,19616.666667,21433.333333,18733.333333,21833.333333,23466.666667,3650.000000
3,Ascent,Bilibili Gaming,4.770000,1092.680000,3.480000,605.780000,4.060000,1.460000,0.600000,0.530000,...,19445.534330,19031.889163,20330.259390,20374.227515,20283.434282,20779.470734,20682.918532,20887.386873,22423.192723,3792.172754
4,Ascent,Cloud9,5.217143,1019.871429,3.694286,670.900000,3.594286,1.665714,0.544286,0.465714,...,17085.714286,20385.714286,23285.714286,20714.285714,24014.285714,23957.142857,21400.000000,21585.714286,22285.714286,4028.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,Split,Team Vitality,5.395000,1043.536667,3.671667,680.643333,3.725000,1.430000,0.580000,0.471667,...,20750.000000,17400.000000,21600.000000,19133.333333,21250.000000,19200.000000,20350.000000,22233.333333,23033.333333,3666.666667
344,Split,Totoro Gaming,5.810000,1146.000000,3.850000,725.300000,3.990000,1.940000,0.600000,0.360000,...,21200.000000,23500.000000,22300.000000,23300.000000,23500.000000,24900.000000,23500.000000,23300.000000,23800.000000,3700.000000
345,Split,Trace Esports,5.910000,969.400000,3.950000,734.980000,3.453333,1.610000,0.580000,0.380000,...,19445.534330,19031.889163,20330.259390,20374.227515,20283.434282,20779.470734,20682.918532,20887.386873,22423.192723,3792.172754
346,Split,Weibo Gaming,5.376667,1098.166667,3.566667,675.253333,3.920000,1.406667,0.580000,0.530000,...,20107.407407,16311.111111,17418.518519,19518.518519,17840.740741,17074.074074,20588.888889,20485.185185,19266.666667,3948.148148


In [76]:
# 'Win' and 'Score diff' are true values from the game, and we will add these teams stats from latest tournaments and the previous tournaments.
# Also, don't use "Player" in test_tournament for convenience.
# Consequently, test_data has 1 less feature ("Player") than stats_team_comp_outcomes (this contains actual values of maps/games in 2023).

test_data = test_tournament[['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team', 'Composition', 'Win','Score_diff']]\
                .merge(data_from_earlier_tnmt, on=["Map", "Team"], how="left")  # Didn't grap "Player" from the features of "test_tournament"

# test_data2 = test_tournament[['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team', 'Composition', 'Win','Score_diff']]\
#                 .merge(data_from_earlier_tnmt2, on=["Map", "Team"], how="left")  # Didn't grap "Player" from the features of "test_tournament"
test_data.isna().sum()

Tournament               0
Stage                    0
Match Type               0
Match Name               0
Map                      0
                        ..
sent_Headshot %          4
cont_Clutch Success %    4
duel_Clutch Success %    4
init_Clutch Success %    4
sent_Clutch Success %    4
Length: 63, dtype: int64

In [77]:
# Same as above cell but with special kill and eco data.

test_data_eco_SK = test_tournament[['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team', 'Composition', 'Win','Score_diff']]\
                .merge(data_from_earlier_tnmt_eco_SK, on=["Map", "Team"], how="left")  # Didn't grap "Player" from the features of "test_tournament"

test_data_eco_SK.isna().sum()

Tournament    0
Stage         0
Match Type    0
Match Name    0
Map           0
             ..
R9_lo         4
R10_lo        4
R11_lo        4
R12_lo        4
R13_lo        4
Length: 79, dtype: int64

In [78]:
# Same as above cell but with special kill rounds and eco data.

test_data_eco_SK_rounds = test_tournament[['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team', 'Composition', 'Win','Score_diff']]\
                .merge(data_from_earlier_tnmt_eco_SK_rounds, on=["Map", "Team"], how="left")  # Didn't grap "Player" from the features of "test_tournament"

test_data_eco_SK_rounds.isna().sum()

Tournament    0
Stage         0
Match Type    0
Match Name    0
Map           0
             ..
R9_lo         4
R10_lo        4
R11_lo        4
R12_lo        4
R13_lo        4
Length: 91, dtype: int64

In [79]:
columns = test_data.keys()
columns_eco_SK = test_data_eco_SK.keys()
columns_eco_SK_rounds =test_data_eco_SK_rounds.keys()

columns_ignore = set(['Tournament', 'Stage', 'Match Type', 'Match Name','Composition', 'Win', 'Score_diff', 'Map', 'Team'])

In [80]:
# Run this cell to fill null values with average of team stats.
for column in columns:
    if column not in columns_ignore:
        test_data[column] = test_data\
                            .drop(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Composition'],axis=1)\
                                .groupby(["Team"])[column].transform(lambda x: x.fillna(x.mean()))

for column in columns_eco_SK:
    if column not in columns_ignore:
        test_data_eco_SK[column] = test_data_eco_SK\
                            .drop(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Composition'],axis=1)\
                                .groupby(["Team"])[column].transform(lambda x: x.fillna(x.mean()))
        
for column in columns_eco_SK_rounds:
    if column not in columns_ignore:
        test_data_eco_SK_rounds[column] = test_data_eco_SK_rounds\
                            .drop(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Composition'],axis=1)\
                                .groupby(["Team"])[column].transform(lambda x: x.fillna(x.mean()))
# for column in columns:
#     if column not in columns_ignore:
#         test_data2[column] = test_data2\
#                             .drop(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Composition'],axis=1)\
#                                 .groupby(["Team"])[column].transform(lambda x: x.fillna(x.mean()))

In [81]:
test_data_eco_SK_rounds.isna().sum().sum()

0

In [82]:
merge_keys3 = ['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A', "Team_A_score_diff", "Team_A_win"]
merge_keys4 = ['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team B', "Team_B_score_diff", "Team_B_win"]

AvsB_test_data =\
    AvsB_test_tournament.merge(
        test_data.rename(columns={"Team":"Team A", "Score_diff":"Team_A_score_diff", "Win":"Team_A_win"})\
            , on=merge_keys3\
                , how="left"\
        )\
        .merge(
        test_data.rename(columns={"Team":"Team B", "Score_diff":"Team_B_score_diff", "Win":"Team_B_win"})\
            , on=merge_keys4\
                , how="left", suffixes=("_A","_B")\
        )\
        
AvsB_test_data_eco_SK =\
    AvsB_test_tournament.merge(
        test_data_eco_SK.rename(columns={"Team":"Team A", "Score_diff":"Team_A_score_diff", "Win":"Team_A_win"})\
            , on=merge_keys3\
                , how="left"\
        )\
        .merge(
        test_data_eco_SK.rename(columns={"Team":"Team B", "Score_diff":"Team_B_score_diff", "Win":"Team_B_win"})\
            , on=merge_keys4\
                , how="left", suffixes=("_A","_B")\
        )\
        
AvsB_test_data_eco_SK_rounds =\
    AvsB_test_tournament.merge(
        test_data_eco_SK_rounds.rename(columns={"Team":"Team A", "Score_diff":"Team_A_score_diff", "Win":"Team_A_win"})\
            , on=merge_keys3\
                , how="left"\
        )\
        .merge(
        test_data_eco_SK_rounds.rename(columns={"Team":"Team B", "Score_diff":"Team_B_score_diff", "Win":"Team_B_win"})\
            , on=merge_keys4\
                , how="left", suffixes=("_A","_B")\
        )\

# AvsB_test_data2 =\
#     AvsB_test_tournament.merge(
#         test_data2.rename(columns={"Team":"Team A", "Score_diff":"Team_A_score_diff", "Win":"Team_A_win"})\
#             , on=merge_keys3\
#                 , how="left"\
#         )\
#         .merge(
#         test_data2.rename(columns={"Team":"Team B", "Score_diff":"Team_B_score_diff", "Win":"Team_B_win"})\
#             , on=merge_keys4\
#                 , how="left", suffixes=("_A","_B")\
#         )\



In [83]:
AvsB_test_data_eco_SK_rounds.isna().sum().sum()

0

**Recall:** We set the following three different type of training data and test data.

```python
train_data_stats_outcomes = stats_team_comp_outcomes[stats_team_comp_outcomes.Tournament != "Valorant Champions 2023"]
train_data_stats_eco_SK_outcomes = stats_eco_and_SK_outcomes[stats_eco_and_SK_outcomes.Tournament != "Valorant Champions 2023"]
train_data_stats_eco_SK_rounds_outcomes = stats_eco_and_SK_rounds_outcomes[stats_eco_and_SK_rounds_outcomes.Tournament != "Valorant Champions 2023"]

test_data
test_data_eco_SK
test_data_eco_SK_rounds
```

In Team A vs Team B format, we have

```python
AvsB_train_stats_outcomes = AvsB_stats_team_comp_outcomes[AvsB_stats_team_comp_outcomes.Tournament != "Valorant Champions 2023"]
AvsB_train_stats_eco_and_SK_outcomes = AvsB_stats_eco_and_SK_outcomes[AvsB_stats_eco_and_SK_outcomes.Tournament != "Valorant Champions 2023"]
AvsB_train_stats_eco_and_SK_rounds_outcomes = AvsB_stats_eco_and_SK_rounds_outcomes[AvsB_stats_eco_and_SK_rounds_outcomes.Tournament != "Valorant Champions 2023"]

AvsB_test_data
AvsB_test_data_eco_SK
AvsB_test_data_eco_SK_rounds
```

In [84]:
print(
train_data_stats_outcomes.isna().sum().sum(),
train_data_stats_eco_SK_outcomes.isna().sum().sum(),
train_data_stats_eco_SK_rounds_outcomes.isna().sum().sum(),

test_data.isna().sum().sum(),
test_data_eco_SK.isna().sum().sum(),
test_data_eco_SK_rounds.isna().sum().sum(),

AvsB_train_stats_outcomes.isna().sum().sum(),
AvsB_train_stats_eco_and_SK_outcomes.isna().sum().sum(),
AvsB_train_stats_eco_and_SK_rounds_outcomes.isna().sum().sum(),

AvsB_test_data.isna().sum().sum(),
AvsB_test_data_eco_SK.isna().sum().sum(),
AvsB_test_data_eco_SK_rounds.isna().sum().sum())

0 0 0 0 0 0 0 0 0 0 0 0


In [85]:
# Sanity check
print(
train_data_stats_outcomes.shape,
train_data_stats_eco_SK_outcomes.shape,
train_data_stats_eco_SK_rounds_outcomes.shape
)
print(
test_data.shape,
test_data_eco_SK.shape,
test_data_eco_SK_rounds.shape
)
print("=================================")
print(
AvsB_train_stats_outcomes.shape,
AvsB_train_stats_eco_and_SK_outcomes.shape,
AvsB_train_stats_eco_and_SK_rounds_outcomes.shape
)
print(
AvsB_test_data.shape,
AvsB_test_data_eco_SK.shape,
AvsB_test_data_eco_SK_rounds.shape
)

(1492, 64) (1492, 80) (1492, 92)
(168, 63) (168, 79) (168, 91)
(746, 123) (746, 155) (746, 179)
(84, 121) (84, 153) (84, 177)


In [86]:
# Sanity check
for key in train_data_stats_outcomes.keys():
    if key not in test_data.keys():
        print(key)

Player


In [87]:
# Sanity check
for key in AvsB_train_stats_outcomes.keys():
    if key not in AvsB_test_data.keys():
        print(key)

Team_A
Team_B
Player_A
Player_B


In [88]:
train_data_stats_outcomes = train_data_stats_outcomes.drop(["Player"], axis=1)
train_data_stats_eco_SK_outcomes = train_data_stats_eco_SK_outcomes.drop(["Player"], axis=1)
train_data_stats_eco_SK_rounds_outcomes = train_data_stats_eco_SK_rounds_outcomes.drop(["Player"], axis=1)

AvsB_train_stats_outcomes = AvsB_train_stats_outcomes.drop(["Player_A", "Player_B"], axis=1)
AvsB_train_stats_eco_and_SK_outcomes = AvsB_train_stats_eco_and_SK_outcomes.drop(["Player_A", "Player_B"], axis=1)
AvsB_train_stats_eco_and_SK_rounds_outcomes = AvsB_train_stats_eco_and_SK_rounds_outcomes.drop(["Player_A", "Player_B"], axis=1)

In [89]:
AvsB_test_data = AvsB_test_data.rename(columns={"Team A": "Team_A", "Team B": "Team_B"})
AvsB_test_data_eco_SK = AvsB_test_data_eco_SK.rename(columns={"Team A": "Team_A", "Team B": "Team_B"})
AvsB_test_data_eco_SK_rounds = AvsB_test_data_eco_SK_rounds.rename(columns={"Team A": "Team_A", "Team B": "Team_B"})

In [90]:
# Save those training and test data above as csv files.
import os

directory = "../data/vct_2023/processed_data"
os.makedirs(directory)

train_data_stats_outcomes.to_csv("../data/vct_2023/processed_data/train_data_players_stats.csv", index=False)
train_data_stats_eco_SK_outcomes.to_csv("../data/vct_2023/processed_data/train_data_players_stats_eco_SK.csv", index=False)
train_data_stats_eco_SK_rounds_outcomes.to_csv("../data/vct_2023/processed_data/train_data_players_stats_eco_SK_rounds.csv", index=False)

test_data.to_csv("../data/vct_2023/processed_data/test_data_players_stats.csv", index=False)
test_data_eco_SK.to_csv("../data/vct_2023/processed_data/test_data_players_stats_eco_SK.csv", index=False)
test_data_eco_SK_rounds.to_csv("../data/vct_2023/processed_data/test_data_players_stats_eco_SK_rounds.csv", index=False)

AvsB_train_stats_outcomes.to_csv("../data/vct_2023/processed_data/AvsB_train_data_players_stats.csv", index=False)
AvsB_train_stats_eco_and_SK_outcomes.to_csv("../data/vct_2023/processed_data/AvsB_train_data_players_stats_eco_SK.csv", index=False)
AvsB_train_stats_eco_and_SK_rounds_outcomes.to_csv("../data/vct_2023/processed_data/AvsB_train_data_players_stats_eco_SK_rounds.csv", index=False)

AvsB_test_data.to_csv("../data/vct_2023/processed_data/AvsB_test_data_players_stats.csv", index=False)
AvsB_test_data_eco_SK.to_csv("../data/vct_2023/processed_data/AvsB_test_data_players_stats_eco_SK.csv", index=False)
AvsB_test_data_eco_SK_rounds.to_csv("../data/vct_2023/processed_data/AvsB_test_data_players_stats_eco_SK_rounds.csv", index=False)
